In [1]:
#importamos la lisbrerias necesarias
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
# Configuración de Selenium con ChromeDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# URL para hacer scraping
url = "https://sgp.corfo.cl/GIN/ActualizacionCentrosID/Views/publico/centros.aspx"

# Abrir la URL con Selenium
driver.get(url)

In [3]:
# Extraer el título, esperando a que los datos se estabilicen
try:
    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#wrapper > div > div:nth-child(1) > div")))
    ante_titulo = ""
    while True:
        titulo = driver.find_element(By.CSS_SELECTOR, "#wrapper > div > div:nth-child(1) > div > h1").text
        if titulo == ante_titulo:
            break
        ante_titulo = titulo
        time.sleep(1)  # Espera un segundo antes de verificar nuevamente
except Exception as e:
    print("No se pudo encontrar el titulo:", e)

print("El titulo es:", titulo)  

El titulo es: CONSULTA CENTROS I+D INSCRITOS REGISTRO CORFO


In [4]:
# Extraer datos de la tabla
try:
    # Encuentra los encabezados de la tabla
    encabezados = [th.text for th in driver.find_elements(By.CSS_SELECTOR, "#grid > thead > tr > th")]

    # Encuentra todas las filas de datos en el cuerpo de la tabla
    filas = driver.find_elements(By.CSS_SELECTOR, "#grid > tbody")

    # Inicializar la lista para almacenar los datos de la tabla
    datos_tabla = []

    # Iterar sobre cada fila para extraer los datos de las celdas
    for fila in filas:
        # Extraer todos los datos de las celdas de esta fila
        datos_fila = [celda.text for celda in fila.find_elements(By.TAG_NAME, "td")]
        
        # Asegurarse de que los datos de la fila se dividan en grupos de 5 para formar las columnas
        for i in range(0, len(datos_fila), 5):
            # Crear una nueva fila con cada grupo de 5 elementos
            fila_dividida = datos_fila[i:i+5]
            # Añadir la fila dividida a la lista de datos de la tabla
            datos_tabla.append(fila_dividida)
except Exception as e:
    print("Error al extraer datos de la tabla:", e)

# Imprimir los datos para verificar
for fila in datos_tabla:
    print(fila)

['SR-001', 'SOCIEDAD ADL DIAGNOSTIC CHILE SPA', 'N/A', 'AGRICULTURA, SILVICULTURA, PESCA, AGROPECUARIAS, BIOTECNOLOGÍA, BIOTECNOLOGÍA AGRÍCOLA, CIENCIA VETERINARIA, CIENCIAS DE LA TIERRA Y CIENCIAS RELACIONADAS CON EL MEDIO AMBIENTE', 'REGIÓN DE LOS LAGOS']
['SR-002', 'UNIVERSIDAD ARTURO PRAT- UNAP', 'UNIVERSIDAD ARTURO PRAT- UNAP', 'AGRICULTURA, SILVICULTURA, PESCA, BIOTECNOLOGÍA, BIOTECNOLOGÍA INDUSTRIAL, OTRAS CIENCIAS NATURALES , PESCA Y ACUICULTURA', 'REGIÓN DE LOS LAGOS']
['SR-003', 'FUNDACIÓN CHILE', 'N/A', 'BIOTECNOLOGÍA MEDIOAMBIENTAL, CIENCIAS BIOLÓGICAS , CIENCIAS DE LA TIERRA Y CIENCIAS RELACIONADAS CON EL MEDIO AMBIENTE , INGENIERÍA MEDIOAMBIENTAL , OTRAS CIENCIAS NATURALES', 'METROPOLITANA DE SANTIAGO']
['SR-004', 'CENTRO INVESTIGACIÓN Y DESARROLLO DE RECURSOS Y AMBIENTES COSTEROS, I-', 'UNIVERSIDAD DE LOS LAGOS', 'BIOTECNOLOGÍA, CIENCIAS BIOLÓGICAS , MEDIOAMBIENTE, PESCA Y ACUICULTURA', 'REGIÓN DE LOS LAGOS']
['SR-005', 'DEPARTAMENTO DE ACUICULTURA Y RECURSOS ACUÁTICOS -

In [5]:
# Crear DataFrame de Pandas con los datos
if datos_tabla:
    df = pd.DataFrame(datos_tabla, columns=encabezados)
    print(df)
else:
    print("No se extrajeron datos de la tabla")

# Cerrar el navegador
driver.quit()

     Código                                             Nombre  \
0    SR-001                  SOCIEDAD ADL DIAGNOSTIC CHILE SPA   
1    SR-002                      UNIVERSIDAD ARTURO PRAT- UNAP   
2    SR-003                                    FUNDACIÓN CHILE   
3    SR-004  CENTRO INVESTIGACIÓN Y DESARROLLO DE RECURSOS ...   
4    SR-005  DEPARTAMENTO DE ACUICULTURA Y RECURSOS ACUÁTIC...   
..      ...                                                ...   
154  SR-260                          FUND CIENCIA PARA LA VIDA   
155  SR-261  CENTRO DE INVESTIGACION Y DESARROLLO TECNOLOGI...   
156  SR-262                 FUNDACIÓN BIONOSTRA CHILE RESEARCH   
157  SR-263  CONSULTORA Y CENTRO DE INVESTIGACIÓN COREDEVX SPA   
158  SR-265                       AMERICAN BIOPROCESS LIMITADA   

                      Patrocinador  \
0                              N/A   
1    UNIVERSIDAD ARTURO PRAT- UNAP   
2                              N/A   
3         UNIVERSIDAD DE LOS LAGOS   
4         UNIVERS

In [6]:
# Eliminar caracteres no permitidos en nombres de archivos
titulo = "".join(c for c in titulo if c.isalnum() or c in (' ', '.', '_')).rstrip()

# Acortar el título si es demasiado largo
max_length = 50  # Puedes ajustar esto según tus necesidades
if len(titulo) > max_length:
    titulo = titulo[:max_length]

# Asegurarse de que el nombre del archivo termine con '.xlsx'
if not titulo.endswith('.xlsx'):
    titulo += '.xlsx'

# Exportar el DataFrame a un archivo Excel
df.to_excel(titulo, index=False)

print("El DataFrame ha sido exportado como", titulo)

El DataFrame ha sido exportado como CONSULTA CENTROS ID INSCRITOS REGISTRO CORFO.xlsx
